# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [18]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [19]:
# Read cities file and store into Pandas data frame
file_to_load = "../WeatherPy/output_data/cities.csv"
cities_df = pd.read_csv(file_to_load)
cities_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,carbonia,39.1672,8.5222,60.80,88,0,3.44,IT,1619382376
1,1,ushuaia,-54.8000,-68.3000,55.40,50,20,16.11,AR,1619382352
2,2,qaanaaq,77.4840,-69.3632,31.48,74,27,4.07,GL,1619382377
3,3,sechura,-5.5569,-80.8222,80.44,58,62,13.42,PE,1619382377
4,4,cape town,-33.9258,18.4232,62.01,94,20,8.05,ZA,1619382377


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [20]:
#Configure gmaps
gmaps.configure(api_key=g_key)

#Use the lat and Lng as locations and humidity as the weight
geolocations = cities_df[["Lat", "Lng"]].astype(float)
humidity = cities_df["Humidity"].astype(float)

In [21]:
#Add Heatmap layer to map
fig = gmaps.figure(center=(20,0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(geolocations, weights=humidity, 
                                 dissipating=False, max_intensity=500,
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [22]:
# Narrow down the DataFrame to find your ideal weather condition. 
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
                           
narrow_df = cities_df.loc[(cities_df["Max Temp"] > 70) & (cities_df["Max Temp"] < 80)
                         & (cities_df["Wind Speed"] < 10)
                         & (cities_df["Cloudiness"] == 0 )]

narrow_df.dropna()
narrow_df
#Total 8 rows returned, reasonable count for api hits

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
97,97,arraial do cabo,-22.9661,-42.0278,77.00,78,0,8.05,BR,1619382396
168,168,beira,-19.8436,34.8389,77.00,83,0,9.95,MZ,1619382411
297,297,zhob,31.3411,69.4481,71.04,30,0,1.45,PK,1619382438
366,366,karratha,-20.7377,116.8463,75.85,62,0,8.46,AU,1619382452
395,395,beloha,-25.1667,45.0500,72.88,79,0,6.71,MG,1619382458
571,571,san juan,-31.5375,-68.5364,73.67,32,0,2.77,AR,1619382494


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
#Store filtered and narrow data frame from above to a new data frame that will include hotel information
#Note: received a SettingWithCopyWarning when simply using the = operator. Using loc and copy per documentation 
hotel_df = narrow_df[0:len(narrow_df)].copy()  
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Address"] = ""

#Set parameters to search for hotels within 5000 meters
params = {
    "radius" : 5000,
    "keyword" : "hotel",
    "key" : g_key
}

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the lat/lng and the rest of the params as set earlier
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params)
    hotels = hotels.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = hotels["results"][0]["vicinity"]
        print(f"Hotel found")
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Hotel found
Hotel found
Hotel found
Hotel found
Missing field/result... skipping.
Hotel found


In [24]:
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address
97,97,arraial do cabo,-22.9661,-42.0278,77.00,78,0,8.05,BR,1619382396,Ocean View Hotel,"Rua, Av. Alfredo Dante Fassini, 01 - Prainha, ..."
168,168,beira,-19.8436,34.8389,77.00,83,0,9.95,MZ,1619382411,Hotel Uzeir,"5R8R+QJ, Beira"
297,297,zhob,31.3411,69.4481,71.04,30,0,1.45,PK,1619382438,Faiz Hotel and Resturant,"Shaheen chowk, 11 Market Rd, Zhob"
366,366,karratha,-20.7377,116.8463,75.85,62,0,8.46,AU,1619382452,Karratha International Hotel,"Corner of Hillview and Millstream Roads, Karratha"
395,395,beloha,-25.1667,45.0500,72.88,79,0,6.71,MG,1619382458,,
571,571,san juan,-31.5375,-68.5364,73.67,32,0,2.77,AR,1619382494,Hotel Viñas del Sol,"San Juan. Ruta 20 Y Gral. Roca, San Juan, Arge..."


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

#Please note, screenshot of the final image included within "output_data/map.png"

Figure(layout=FigureLayout(height='420px'))